# Seqeuence Alignment

Mojo vs Python for performing local sequence alignment via the [Smith–Waterman algorithm](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm) between two arrays.

## Python

In [1]:
%%python  

# Import necessary libraries
import time
import numpy as np
from timeit import timeit

# Define the size of the arrays
n = 1_000

# Generate a random array 'anp' of integers between 0 and 3, of size 'n'
anp = np.random.randint(0, 4, size=n)  # This will generate random integers between 0 and 3 inclusive

# Generate a random array 'bnp' of integers between 0 and 3, of size 'n'
bnp = np.random.randint(0, 4, size=n)  # This will generate random integers between 0 and 3 inclusive

# Define a function 'print_formatter' to format and print a string and a value
def print_formatter(string, value):
    print(f"{string}: {value:5.5f}")

# Comment describing the current state of the arrays 'anp' and 'bnp'
# Now, anp and bnp are arrays of random integers between 0 and 3.


### Analysis

In [2]:
%%python 

# Define the Smith-Waterman algorithm function with parameters for match score, mismatch penalty, and gap penalty
def smith_waterman_py(anp, bnp, match_score=1, mismatch_penalty=-1, gap_penalty=-1):
    
    # Determine the dimensions of the score matrix
    rows = len(anp) + 1
    cols = len(bnp) + 1
    
    # Initialize the score matrix with zeros
    score_matrix = np.zeros((rows, cols), dtype=int)
    
    # Initialize variables to keep track of the maximum score and its position
    max_score = 0
    max_pos = None
    
    # Populate the score matrix by iterating through each cell
    for i in range(1, rows):
        for j in range(1, cols):
            
            # Calculate the score for a diagonal move (match or mismatch)
            score_diag = score_matrix[i-1, j-1] + (match_score if anp[i-1] == bnp[j-1] else mismatch_penalty)
            
            # Calculate the score for a move to the left (gap in sequence 'anp')
            score_left = score_matrix[i, j-1] + gap_penalty
            
            # Calculate the score for a move up (gap in sequence 'bnp')
            score_up = score_matrix[i-1, j] + gap_penalty
            
            # Determine the maximum score among the three moves or zero, and update the score matrix
            score_matrix[i, j] = max(0, score_diag, score_left, score_up)
            
            # Update the maximum score and its position if a higher score is found
            if score_matrix[i, j] >= max_score:
                max_score = score_matrix[i, j]
                max_pos = (i, j)
    
    # Initialize variables to store the aligned sequences
    align1, align2 = '', ''
    i, j = max_pos  # Start at the position of the maximum score
    
    # Trace back from the maximum score position to the origin, constructing the aligned sequences
    while i > 0 and j > 0 and score_matrix[i, j] != 0:
        
        # Recalculate the score for a diagonal move
        score_diag = score_matrix[i-1, j-1] + (match_score if anp[i-1] == bnp[j-1] else mismatch_penalty)
        
        # Determine the move that resulted in the current score, and update the aligned sequences and indices accordingly
        if score_matrix[i, j] == score_diag:
            align1 = str(anp[i-1]) + align1
            align2 = str(bnp[j-1]) + align2
            i -= 1
            j -= 1
        elif score_matrix[i, j] == score_matrix[i, j-1] + gap_penalty:
            align1 = '-' + align1
            align2 = str(bnp[j-1]) + align2
            j -= 1
        else:
            align1 = str(anp[i-1]) + align1
            align2 = '-' + align2
            i -= 1
    
    # Return the maximum score after completing the trace back
    return max_score

# Time the execution of the Smith-Waterman algorithm using 'timeit'
secs = timeit(lambda: smith_waterman_py(anp, bnp), number=5)/5

# Print the performance of the Smith-Waterman algorithm
print("=== Smith-Waterman Performance ===")
alignment = smith_waterman_py(anp, bnp)  # Run the Smith-Waterman algorithm and store the maximum score
print("Max Score:", alignment)  # Print the maximum score
print_formatter("Smith-Waterman time (ms)", 1000 * secs)  # Print the execution time in milliseconds

=== Smith-Waterman Performance ===
Max Score: 109
Smith-Waterman time (ms): 840.64388


## Mojo

### Create Tensors

In [3]:
# Load Mojo librarires - some may not be needed for this notebook

from algorithm import vectorize, parallelize
from benchmark import Benchmark
from math import div_ceil, min, sqrt, floor, mod, max
from memory import memset_zero
from memory.buffer import Buffer
from memory.unsafe import DTypePointer
from random import rand, randint, random_float64
from runtime.llcl import Runtime, num_cores
from sys.intrinsics import strided_load
from sys.info import simdwidthof
from tensor import Tensor
from time import now
from utils.list import VariadicList

In [4]:
# Declare an integer 'n' and set it to one thousand
let n: Int = 1_000  

# Create a tensor 'x' of int64 data type with 'n' elements
var x = Tensor[DType.int64](n)  

# Create a tensor 'y' of int64 data type with 'n' elements
var y = Tensor[DType.int64](n)  

# Fill tensor 'x' with random integers between 0 and 3
randint[DType.int64](x.data(), n, 0, 3)  

# Fill tensor 'y' with random integers between 0 and 3
randint[DType.int64](y.data(), n, 0, 3)  


### Create Matrix Struct

In [5]:
# Define aliases for different pointer types
alias PointerString = Pointer[UInt8]
alias BufferPtrType = DTypePointer[DType.uint8]
alias BufferPtrFloat32 = DTypePointer[DType.float32]
alias PointerStrings = Pointer[PointerString]

# Define a Matrix struct with methods for initialization, allocation, and data access
struct Matrix:
    # Declare members of the Matrix struct: data pointer, dimensions, and allocation flag
    var data: BufferPtrFloat32
    var rows: Int
    var cols: Int
    var layers: Int
    var allocated: Int

    # Overloaded constructor for 2D matrix initialization
    fn __init__(inout self, rows: Int, cols: Int):
        self.data = BufferPtrFloat32.alloc(0)  # Initialize data pointer to null
        self.rows = rows  # Set rows
        self.cols = cols  # Set columns
        self.layers = 1  # Default to 1 layer
        self.allocated = 0  # Set allocation flag to false

    # Overloaded constructor for 1D matrix (vector) initialization
    fn __init__(inout self, cols: Int):
        self.data = BufferPtrFloat32.alloc(0)  # Initialize data pointer to null
        self.rows = 1  # Default to 1 row
        self.layers = 1  # Default to 1 layer
        self.cols = cols  # Set columns
        self.allocated = 0  # Set allocation flag to false

    # Overloaded constructor for 3D matrix initialization
    fn __init__(inout self, layers: Int, rows: Int, cols: Int):
        self.__init__(rows, cols)  # Call 2D constructor
        self.layers = layers  # Set layers

    # Destructor to free allocated data
    fn __del__(owned self):
        if self.allocated == 1:  # Check if data is allocated
            self.data.free()  # Free allocated data

    # Method to allocate data for the matrix
    @always_inline
    fn alloc(inout self, fill: Int = 0):
        self.data = BufferPtrFloat32.alloc(self.size())  # Allocate data
        self.allocated = 1  # Set allocation flag to true
        if fill == 1:  # Check if fill flag is set
            self.zero()  # Zero-fill the allocated data

    # Method to allocate and zero-fill data for the matrix
    @always_inline
    fn alloc_zero(inout self):
        self.alloc(1)  # Call alloc with fill flag set

    # Method to zero-fill the matrix data
    @always_inline
    fn zero(inout self):
        memset_zero(self.data, self.size())  # Zero-fill the data

    # Method to set the data pointer of the matrix
    @always_inline
    fn set_buf_ptr(inout self, ptr: BufferPtrFloat32):
        self.data = ptr  # Set data pointer

    # Method to set the data pointer and redefine dimensions of the matrix
    fn set_buf_ptr(inout self, ptr: BufferPtrFloat32, rows: Int, cols: Int):
        self.data = ptr  # Set data pointer
        self.rows = rows  # Set rows
        self.cols = cols  # Set columns

    # Method to calculate the size (number of elements) of the matrix
    @always_inline
    fn size(inout self) -> Int:
        return self.cols * self.rows * self.layers  # Calculate size

    # Overloaded indexing operator for 2D access
    @always_inline
    fn __getitem__(self, y: Int, x: Int) -> Float32:
        return self.load[1](y, x)  # Load single element

    # Overloaded indexing operator for 1D access
    @always_inline
    fn __getitem__(self, x: Int) -> Float32:
        return self.load[1](0, x)  # Load single element

    # SIMD load method for 2D access
    @always_inline
    fn load[nelts: Int](self, y: Int, x: Int) -> SIMD[DType.float32, nelts]:
        return self.data.simd_load[nelts](y * self.cols + x)  # Load SIMD vector

    # Overloaded assignment operator for 2D access
    @always_inline
    fn __setitem__(self, y: Int, x: Int, val: Float32):
        return self.store[1](y, x, val)  # Store single element

    # Overloaded assignment operator for 1D access
    @always_inline
    fn __setitem__(self, x: Int, val: Float32):
        return self.store[1](0, x, val)  # Store single element

    # SIMD store method for 2D access
    @always_inline
    fn store[nelts: Int](self, y: Int, x: Int, val: SIMD[DType.float32, nelts]):
        self.data.simd_store[nelts](y * self.cols + x, val)  # Store SIMD vector

    # SIMD load method for 1D access
    @always_inline
    fn load[nelts: Int](self, x: Int) -> SIMD[DType.float32, nelts]:
        return self.data.simd_load[nelts](x)  # Load SIMD vector

    # SIMD store method for 1D access
    @always_inline
    fn store[nelts: Int](self, x: Int, val: SIMD[DType.float32, nelts]):
        self.data.simd_store[nelts](x, val)  # Store SIMD vector

    # Overloaded indexing operator for 3D access
    @always_inline
    fn __getitem__(self, z: Int, y: Int, x: Int) -> Float32:
        return self.load[1](z, y, x)  # Load single element

    # SIMD load method for 3D access
    @always_inline
    fn load[nelts: Int](self, z: Int, y: Int, x: Int) -> SIMD[DType.float32, nelts]:
        return self.data.simd_load[nelts](z * self.layers + y * self.cols + x)  # Load SIMD vector

    # Overloaded assignment operator for 3D access
    @always_inline
    fn __setitem__(self, z: Int, y: Int, x: Int, val: Float32):
        return self.store[1](z, y, x, val)  # Store single element

    # SIMD store method for 3D access
    @always_inline
    fn store[nelts: Int](self, z: Int, y: Int, x: Int, val: SIMD[DType.float32, nelts]):
        self.data.simd_store[nelts](z * self.layers + y * self.cols + x, val)  # Store SIMD vector


### Typed Approach

In [6]:
# Define the Smith-Waterman function for sequence alignment
fn smith_waterman(x: Tensor[DType.int64], y: Tensor[DType.int64], match_score: Float32, mismatch_penalty: Float32, gap_penalty: Float32) -> Float32:
    # Determine the dimensions of the score matrix
    let rows = x.num_elements() + 1
    let cols = y.num_elements() + 1
    
    # Create a Matrix instance for the score matrix and allocate zero-initialized memory for it
    var score_matrix = Matrix(rows, cols)
    score_matrix.alloc_zero()
    
    # Initialize variables to keep track of the maximum score and its position
    var max_score: Float32 = 0
    var max_i: Int = 0  # Changed line
    var max_j: Int = 0  # Changed line
    
    # Populate the score matrix by iterating through each cell
    for i in range(1, rows):
        for j in range(1, cols):
            # Determine the score for a diagonal move (match or mismatch)
            let score_diag: Float32
            if x[i-1] == y[j-1]:
                score_diag = score_matrix[i-1, j-1] + match_score
            else:
                score_diag = score_matrix[i-1, j-1] + mismatch_penalty
            
            # Determine the score for a move to the left (gap in sequence 'x')
            let score_left = score_matrix[i, j-1] + gap_penalty
            # Determine the score for a move up (gap in sequence 'y')
            let score_up = score_matrix[i-1, j] + gap_penalty
            
            # Determine the local maximum score among the three moves or zero, and update the score matrix
            let local_max = max(0, max(score_diag, max(score_left, score_up)))
            score_matrix[i, j] = local_max
            
            # Update the maximum score and its position if a higher score is found
            if local_max >= max_score:
                max_score = local_max
                max_i = i  # Changed line
                max_j = j  # Changed line
    
    # Initialize strings to store the aligned sequences
    var align1: String = ""
    var align2: String = ""
    # Set indices to the position of the maximum score for traceback
    var i = max_i  # Changed line
    var j = max_j  # Changed line
    # Trace back from the position of the maximum score to the origin, constructing the aligned sequences
    while i > 0 and j > 0 and score_matrix[i, j] != 0:
        # Recalculate the score for a diagonal move
        let score_diag: Float32
        if x[i-1] == y[j-1]:
            score_diag = score_matrix[i-1, j-1] + match_score
        else:
            score_diag = score_matrix[i-1, j-1] + mismatch_penalty
        
        # Determine the move that resulted in the current score, and update the aligned sequences and indices accordingly
        if score_matrix[i, j] == score_diag:
            align1 = String(x[i-1]) + align1
            align2 = String(y[j-1]) + align2
            i -= 1
            j -= 1
        elif score_matrix[i, j] == score_matrix[i, j-1] + gap_penalty:
            align1 = '-' + align1
            align2 = String(y[j-1]) + align2
            j -= 1
        else:
            align1 = String(x[i-1]) + align1
            align2 = '-' + align2
            i -= 1
    
    # Return the maximum score after completing the trace back
    return max_score


In [7]:
# Capture the start time before execution
var eval_begin = now()

# Execute the Smith-Waterman function and store the result in max_score_result
# Parameters: match score = 1.0, mismatch penalty = -1.0, gap penalty = -1.0
var max_score_result = smith_waterman(x, y, 1.0, -1.0, -1.0)  # Renamed variable for clarity

# Capture the end time after execution
var eval_end = now()

# Print the result of the Smith-Waterman function
print("Max Score: ", max_score_result)

# Calculate and print the execution time in milliseconds
print("Execution Time: ", Float64((eval_end - eval_begin)) / 1e6, "ms")

# Calculate and print the speedup factor compared to a reference time (secs)
print("Speedup", (1000*secs) / (Float64((eval_end - eval_begin)) / 1e6))


Max Score:  103.0
Execution Time:  3.2947479999999998 ms
Speedup 255.1466388931721
